**Tarea 5 Diego Saul Andrade Juarez**

In [ ]:
# Instalar las bibliotecas necesarias
install.packages(c("knitr", "tidyverse", "discrtr", "dplyr", "ggplot2",
                   "mlogit", "readr", "gplots", "ggmosaic",
                   "treemapify", "ggridges", "ggalluvial",
                   "evd", "htmlwidgets", "kableExtra",
                   "plotly", "tidyr"))

# Cargar las bibliotecas
library(knitr)
library(tidyverse)
library(discrtr) # A companion package for the book Introduction to Discrete Choice Analysis with `R`
library(dplyr) # A Grammar of Data Manipulation
library(ggplot2) # Create Elegant Data Visualisations Using the Grammar of Graphics
library(mlogit) # Multinomial Logit Models
library(readr) # Read Rectangular Text Data
library(gplots) # Various R Programming Tools for Plotting Data
library(ggmosaic) # Mosaic Plots in the 'ggplot2' Framework
library(treemapify)
library(ggridges)
library(ggalluvial)
library(evd)
library(htmlwidgets) # HTML Widgets for R
library(kableExtra) # Construct Complex Table with kable and Pipe Syntax
library(plotly) # Create interactive web graphics
library(tidyr) # Tidy messy data
# library(webshot2) # Take screenshots of web pages

# Configurar opciones de knitr
knitr::opts_chunk$set(echo = TRUE)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘litedown’, ‘markdown’, ‘png’, ‘jpeg’, ‘bookdown’, ‘rbibutils’, ‘bitops’, ‘plyr’, ‘shades’, ‘gridtext’, ‘rmdformats’, ‘dfidx’, ‘Formula’, ‘zoo’, ‘lmtest’, ‘statmod’, ‘Rdpack’, ‘gtools’, ‘caTools’, ‘productplots’, ‘ggrepel’, ‘ggfittext’, ‘lazyeval’, ‘svglite’, ‘crosstalk’


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: dfidx


Attaching package: ‘dfidx’


The following object i

# **1. ¿Cuál es la diferencia entre un efecto marginal y una elasticidad?**

*El efecto marginal muestra cuánto cambia una probabilidad si una variable aumenta un poco, mientras que la elasticidad muestra el cambio porcentual en la probabilidad frente a un cambio porcentual en la variable.*

# **2. ¿Por qué es inapropiado calcular la elasticidad de una variable dicotómica (dummy)?**

*No se puede calcular elasticidad en una variable dummy porque solo toma valores 0 o 1, y los cambios porcentuales no se pueden medir en estos casos. En su lugar, se usan efectos marginales.*

# **3. ¿Cuáles son los efectos marginales y elasticidades del costo operativo en la media de todas las variables?**

In [ ]:
data("Heating")

H <- mlogit.data(Heating, shape = "wide", choice = "depvar", varying = 3:12)
model3 <- mlogit(depvar ~ ic + oc, data = H, reflevel = "ec")

medias <- Heating[1, ]
medias[3:12] <- Heating %>% select(starts_with("ic"), starts_with("oc")) %>% summarise(across(everything(), mean))


for (type in c("ra", "rr")) {
  result <- effects(model3, covariate = "oc", type = type, data = mlogit.data(medias, shape = "wide", choice = "depvar", varying = 3:12))
  print(paste("Resultados para tipo:", type))
  print(result)
}

[1] "Resultados para tipo: ra"
              ec            er            gc            gr            hp
ec -0.0065244800  0.0004718878  0.0004718878  0.0004718877  0.0004718878
er  0.0006241011 -0.0063722666  0.0006241011  0.0006241011  0.0006241011
gc  0.0045085704  0.0045085704 -0.0024877975  0.0045085704  0.0045085704
gr  0.0010045002  0.0010045002  0.0010045002 -0.0059918675  0.0010045002
hp  0.0003873084  0.0003873084  0.0003873084  0.0003873084 -0.0066090592
[1] "Resultados para tipo: rr"
            ec          er          gc          gr         hp
ec -3.11089169  0.22499751  0.22499750  0.22499750  0.2249975
er  0.26819487 -2.73835326  0.26819487  0.26819487  0.2681949
gc  0.77599637  0.77599637 -0.42818936  0.77599637  0.7759964
gr  0.15516657  0.15516657  0.15516657 -0.92557229  0.1551666
hp  0.08493646  0.08493646  0.08493646  0.08493646 -1.4493621


# **4. ¿Qué subsidio se necesita para que las bombas de calor tengan 10% de adopción?**

*El subsidio necesario es de $450 unidades monetarias para lograr que el 10% de los hogares elijan bombas de calor, segunún lo que nos dice el modelo*

In [ ]:
descuentos <- seq(0, 2000, by = 50)

hp_shares <- numeric(length(descuentos))


Heating_original <- Heating
for (i in seq_along(descuentos)) {
  Heating_temp <- Heating_original
  Heating_temp$ic.hp <- Heating_temp$ic.hp - descuentos[i]

prob_temp <- predict(model3, newdata = mlogit.data(Heating_temp, shape = "wide", choice = "depvar", varying = 3:12, id.var = "idcase"), type = "probabilities")

  hp_shares[i] <- mean(prob_temp[, "hp"])}


required_descuento <- descuentos[which(hp_shares >= 0.10)[1]]
required_descuento

[1] 450

# **4.1 Estima un nuevo modelo con la edad del jefe del hogar y referencia “er”.**

In [ ]:
model4 <- mlogit(depvar ~ ic + oc | agehed, Heating, shape = "wide", choice = "depvar", reflevel = "er", varying = c(3:12))

summary(model4)


Call:
mlogit(formula = depvar ~ ic + oc | agehed, data = Heating, reflevel = "er", 
    shape = "wide", choice = "depvar", varying = c(3:12), method = "nr")

Frequencies of alternatives:choice
      er       ec       gc       gr       hp 
0.093333 0.071111 0.636667 0.143333 0.055556 

nr method
6 iterations, 0h:0m:0s 
g'(-H)^-1g = 3.7E-05 
successive function values within tolerance limits 

Coefficients :
                  Estimate  Std. Error z-value  Pr(>|z|)    
(Intercept):ec -1.23573437  0.54579124 -2.2641 0.0235670 *  
(Intercept):gc -1.00448043  0.53688731 -1.8709 0.0613543 .  
(Intercept):gr -2.57939938  0.61056437 -4.2246 2.393e-05 ***
(Intercept):hp -2.10149168  0.63311233 -3.3193 0.0009024 ***
ic             -0.00152526  0.00062225 -2.4512 0.0142385 *  
oc             -0.00703320  0.00155731 -4.5163 6.294e-06 ***
agehed:ec       0.02501028  0.01201972  2.0808 0.0374550 *  
agehed:gc       0.02070566  0.00858773  2.4111 0.0159056 *  
agehed:gr       0.02462347  0.01024474  

# **6. Compara el nuevo modelo con el Modelo 3 con una prueba de razón de verosimilitud.**

In [ ]:
lrtest(model3, model4)

,#Df,LogLik,Df,Chisq,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,6,-1008.229,NA,NA,NA
2,10,-1003.772,4,8.913225,0.06330541


# **6. ¿Es significativa la razón entre el coeficiente del coste de instalación (u operación) y el coeficiente de la edad del jefe de familia? Explique.**

*No se puede intrerpretar ños valores de los coeficientes ya que son distintos tipos de variables uno es especifico de la algernativa y otra del individuo*